In [ ]:
! pip install transformers datasets
! pip install rouge-score nltk
! pip install huggingface_hub

# Model used for finetuning "Falconsai/medical_summarization"

# Data set used "starmpcc/Asclepius-Synthetic-Clinical-Notes"

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# !apt install git-lfs
# !git config --global user.email "saurabh.gupta18891@gmail.com"
# !git config --global user.name "Saurabh91"

Make sure your version of Transformers is at least 4.16.0 since some of the functionality we use was introduced in that version:

In [ ]:
import transformers
print(transformers.__version__)

4.35.2


In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("summarization_notebook", framework="tensorflow")

# Fine-tuning a model on a summarization task

In [ ]:
# model_checkpoint = "t5-small"

In [ ]:
model_checkpoint = "Falconsai/medical_summarization"

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("starmpcc/Asclepius-Synthetic-Clinical-Notes",split='train[110000:]')

In [ ]:
raw_datasets

Dataset({
    features: ['patient_id', 'note', 'question', 'answer', 'task'],
    num_rows: 48114
})

In [ ]:
raw_datasets = raw_datasets.filter(lambda example: "summarization" in example["task"].lower())

In [ ]:
raw_datasets

Dataset({
    features: ['patient_id', 'note', 'question', 'answer', 'task'],
    num_rows: 5929
})

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
raw_datasets=raw_datasets.shuffle(seed=666)

In [ ]:
raw_datasets

Dataset({
    features: ['patient_id', 'note', 'question', 'answer', 'task'],
    num_rows: 5929
})

In [ ]:
from datasets import *
# ds = load_dataset("myusername/mycorpus")

train_testvalid = raw_datasets.train_test_split(test_size=0.2)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
raw_datasets = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'note', 'question', 'answer', 'task'],
        num_rows: 4743
    })
    test: Dataset({
        features: ['patient_id', 'note', 'question', 'answer', 'task'],
        num_rows: 593
    })
    valid: Dataset({
        features: ['patient_id', 'note', 'question', 'answer', 'task'],
        num_rows: 593
    })
})

To access an actual element, you need to select a split first, then give an index:

In [ ]:
raw_datasets["train"]["note"][:2]

["Discharge Summary:\n\nPatient: 48-year-old man with rectosigmoid colon cancer\nHospital Stay Dates: N/A (Hospital Course Summary)\n\nReason for Hospitalization:\nImplantable venous port removal via percutaneous approach through the right common femoral artery due to the fractured catheter of the removed right subclavian venous port and its subsequent embolization.\n\nHospital Course:\nThe patient was admitted to the hospital for the removal of an implanted venous port that had been inserted through the right subclavian vein for adjuvant chemotherapy treatment. The location of the catheter was confirmed by chest radiography. After completing four rounds of chemotherapy, the patient ended the therapy of his own accord and was lost to follow-up. He returned to the outpatient clinic six months after terminating the therapy, and chemotherapy was initiated again with different medications. However, the venous catheter became dysfunctional after the third cycle, necessitating the removal of

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

,patient_id,note,question,answer,task
0,149559,"Hospital Course:\nThe 70-year-old male patient was admitted for elective coronary artery bypass grafting due to Type 2 diabetes, chronic kidney disease, hiatus hernia, and normochromic normocytic anemia. The procedure was uneventful, and the patient was taken off bypass with minimal inotropic support. On the first post-operative day, the patient required a resternotomy due to a bleeding point from the IMA pedicle. The patient's recovery was noted to be slow but steady. However, on the tenth day after surgery, the patient developed signs of tamponade and underwent an emergency resternotomy that revealed a dehisced LIMA-LAD anastomosis. Salvaging procedures were successfully performed, and the patient was extubated after a second ventilator wean. The patient was discharged 42 days later and was alive and well with no chest pain after a 5-year follow-up. \n\nHospitalization Summary:\nA 70-year-old male patient was admitted to the hospital for elective coronary artery bypass grafting. The patient had a complicated medical history that included chronic kidney disease, Type 2 diabetes, hiatus hernia, normochromic normocytic anemia, and BMI. The procedure was uncomplicated, and the patient was taken off bypass with minimal inotropic support. However, the patient required a resternotomy due to a bleeding point from the IMA pedicle on the first post-operative day. Recovery was slow but steady until the patient developed signs of tamponade on the tenth day. An emergency resternotomy was performed, revealing a dehisced LIMA-LAD anastomosis. Salvaging procedures were performed, and the patient was discharged on the 42nd post-operative day. At the five-year follow-up, the patient was alive and well with no chest pain.",What were the complications and salvaging procedures involved in a 70-year-old male patient's emergency resternotomy following signs of tamponade 10 days after uncomplicated coronary artery bypass grafting? Can you summarize them?,"The patient's emergency resternotomy was performed due to signs of tamponade and revealed a dehisced LIMA-LAD anastomosis. Salvaging procedures were performed successfully, and the patient was discharged on the 42nd post-operative day. Unfortunately, the discharge summary did not provide specific details regarding the complications and salvaging procedures involved in the patient's emergency resternotomy.",Summarization
1,159156,"Patient Information:\nName: Confidential\nAge: 24\nGender: Male\nMedical Record Number: Confidential\nAdmission Date: Confidential\nDischarge Date: Confidential\n\nHospital Course:\nThe patient was admitted to the hospital due to a persistent systemic erythema and wheals, accompanied by a stinging pain with a visual analogue scale (VAS) rating of 7 cm. The symptoms were aggravated when the patient engaged in physical activities, consumed hot or spicy foods, or experienced situational nervousness or stress. The symptoms were relieved within 30-90 minutes of rest or symptomatic relief. The patient also suffered from excessive hot flushes when exposed to heat.\n\nDiagnostic Findings:\nThe blood laboratory test, liver function test, urinalysis, rheumatoid factor, coagulation test, Erythrocyte Sedimentation Rate (ESR), C-reactive protein (CRP), and chest X-ray were all within normal ranges or negative. Glycopyrrolate 2 mg/day was administered orally for 1 week for diagnostic purposes, but the patient still complained of severe pain and hot flushes.\n\nTreatment:\nA right and left stellate ganglion block was carried out by administering Mepivacaine 10 cc with a 20-minute interval. The patient showed a reduction in the degree of excessive hot flushes, and the VAS rating of the systemic pain was reduced from 7 cm to 2 cm.\n\nFollow-up:\nThe patient was discharged after a follow-up period of 1 week. The frequency of symptoms was similar, but there was a reduction in the degree of excessive hot flushes, and the systemic pain had a VAS

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

## Preprocessing the data

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [8774, 6, 48, 19, 3, 9, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 19, 3, 9, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this is a sentence!", "This is another sentence."]))

In [ ]:
if model_checkpoint in ["t5-small", "t5-large","Falconsai/medical_summarization"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
max_input_length = 1024
max_target_length = 128


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["note"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["answer"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(raw_datasets["train"][:2])

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/4743 [00:00<?, ? examples/s]

Map:   0%|          | 0/593 [00:00<?, ? examples/s]

Map:   0%|          | 0/593 [00:00<?, ? examples/s]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is sequence-to-sequence (both the input and output are text sequences), we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = "Saurabh91/medical_summarization-finetuned-starmpccAsclepius-Synthetic-Clinical-Notes"

In [ ]:
model_name = model_checkpoint.split("/")[-1]

In [ ]:
model_name

'medical_summarization'

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np")

generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np", pad_to_multiple_of=128)

In [ ]:
tokenized_datasets["train"]

Dataset({
    features: ['patient_id', 'note', 'question', 'answer', 'task', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 4743
})

In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["valid"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator
)

In [ ]:
from transformers import AdamWeightDecay
import tensorflow as tf

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback, KerasMetricCallback
from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir="./summarization_model_save/logs")

push_to_hub_callback = PushToHubCallback(
    output_dir="./summarization_model_save",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
)

# # metric_callback = KerasMetricCallback(
#     metric_fn, eval_dataset=generation_dataset, predict_with_generate=True, use_xla_generation=True
# )

callbacks = [tensorboard_callback, push_to_hub_callback]

model.fit(
    train_dataset, validation_data=validation_dataset, epochs=1, callbacks=callbacks
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/summarization_model_save is already a clone of https://huggingface.co/Saurabh91/medical_summarization-finetuned-starmpccAsclepius-Synthetic-Clinical-Notes. Make sure you pull the latest changes with `repo.git_pull()`.


  6/592 [..............................] - ETA: 3:57 - loss: 0.9790

592/592 [==============================] - 324s 546ms/step - loss: 0.9433 - val_loss: 0.8270


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Inference

Now we've trained our model, let's see how we could load it and use it to summarize text in future! First, let's load it from the hub. This means we can resume the code from here without needing to rerun everything above every time.

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

# You can of course substitute your own username and model here if you've trained and uploaded it!
model_name = 'Saurabh91/medical_summarization-finetuned-starmpccAsclepius-Synthetic-Clinical-Notes'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/374M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at Saurabh91/medical_summarization-finetuned-starmpccAsclepius-Synthetic-Clinical-Notes.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [ ]:
document='''Hospital Course Summary:

Admission Date: [Date]
Discharge Date: [Date]
Length of Stay: 24 days

Admitting Diagnosis: Severe malnutrition

Hospital Course:
The patient was admitted with bradycardia and hypoglycaemia with a capillary blood glucose level of 2.3 mmol/L. He had a history of eating and anxiety disorders and was severely malnourished. His blood pressure was initially un-recordable but subsequently was recorded to be 104/72 mmHg, and his capillary blood glucose level improved to 4.7 mmol/L. He was noted to be cachexic and had abnormal liver function tests. His condition improved over the course of his 24-day admission, but he had a number of hypoglycaemic events and difficulty complying with treatment. Due to problems keeping him compliant with medication and intravenous glucose, he was placed under Mental Health Act 5(2) and was started on nasogastric feeding. His liver synthetic function improved over the course of his stay, but there were some concerns about nasogastric feeding, which led to worsening LFTs. He was discharged to an Eating Disorders Unit.

Discharge Diagnosis: Severe malnutrition, cachexia

Disposition: Discharged to an Eating Disorders Unit

Follow-up Plan: The patient was provided with a referral to the Eating Disorders Unit for continued care and management of his severe malnutrition and cachexia.

Course of Treatment:
- The patient was initially treated with intravenous glucose to address hypoglycaemia and rehydrate. He was subsequently started on nasogastric feeding to address his severe malnutrition and undernourishment.
- His liver function tests worsened with the introduction of nasogastric feeding, but a non-invasive liver screen and ultrasound provided reassurance.
- The patient's condition, LFTs, and liver synthetic function improved over the course of his admission.
- The patient's compliance with treatment was challenging, and he required Mental Health Act 5(2) to ensure his continued care and management.
- The patient was discharged to an Eating Disorders Unit for further care and management of his severe malnutrition and cachexia.

Discharge Medications:
None'''



In [ ]:
if 'medical_summarization' in model_name:
    document = "summarize: " + document
tokenized = tokenizer([document], return_tensors='np')
out = model.generate(**tokenized, max_length=512)

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0]))

<pad> The patient was diagnosed with severe malnutrition and cachexia, and was initially treated with intravenous glucose to address hypoglycaemia and rehydrate, but a non-invasive liver screen and ultrasound provided reassurance. The patient was discharged to an Eating Disorders Unit for further care and management of his severe malnutrition and cachexia. The patient was prescribed nasogastric feeding, and the patient was discharged to an Eating Disorders Unit for further care and management of his severe malnutrition and cachexia.</s>


In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

The patient was diagnosed with severe malnutrition and cachexia, and was initially treated with intravenous glucose to address hypoglycaemia and rehydrate, but a non-invasive liver screen and ultrasound provided reassurance. The patient was discharged to an Eating Disorders Unit for further care and management of his severe malnutrition and cachexia. The patient was prescribed nasogastric feeding, and the patient was discharged to an Eating Disorders Unit for further care and management of his severe malnutrition and cachexia.


## Pipeline API

In [ ]:
from transformers import pipeline

summarizer = pipeline('text2text-generation', model_name, framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at Saurabh91/medical_summarization-finetuned-starmpccAsclepius-Synthetic-Clinical-Notes.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
generated_summary=summarizer(document, max_length=512)

Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
generated_summary

[{'generated_text': 'The patient was diagnosed with severe malnutrition and cachexia, and was initially treated with intravenous glucose to address hypoglycaemia and rehydrate, but a non-invasive liver screen and ultrasound provided reassurance. The patient was discharged to an Eating Disorders Unit for further care and management of his severe malnutrition and cachexia. The patient was prescribed nasogastric feeding, and the patient was discharged to an Eating Disorders Unit for further care and management of his severe malnutrition and cachexia.'}]

In [ ]:
generated_summary[0]["generated_text"]

'The patient was diagnosed with severe malnutrition and cachexia, and was initially treated with intravenous glucose to address hypoglycaemia and rehydrate, but a non-invasive liver screen and ultrasound provided reassurance. The patient was discharged to an Eating Disorders Unit for further care and management of his severe malnutrition and cachexia. The patient was prescribed nasogastric feeding, and the patient was discharged to an Eating Disorders Unit for further care and management of his severe malnutrition and cachexia.'